In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings
import json
import time
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import copy

warnings.filterwarnings("ignore")

FISSION_URL = 'http://172.26.135.52:9090/'
FISSION_HEADERS = {'HOST': 'fission'}

In [2]:
def get_full_data(params):
    # Define search query (optional, can be empty to retrieve all documents)
    data = []
    max_retries = 3
    retry_delay = 5  # seconds
    timeout = 60  # seconds

    for _ in range(max_retries):
        try:
            res = requests.get(f"{FISSION_URL}/{params}", headers=FISSION_HEADERS, timeout=timeout)
            if res.status_code != 200:
                print(res.text)
                return None
            data = json.loads(res.text)
            return data
        except requests.exceptions.RequestException as e:
            print(f"Connection error: {e}")
            print("Retrying in 5 seconds...")
            time.sleep(retry_delay)
    
    print("Max retries exceeded. Unable to retrieve data.")
    return None

In [3]:

resp = get_full_data('stations')
if resp is None:
    print("Failed to retrieve data.")
elif resp['Status'] != 200:
    print(f"Failed to retrieve data. Status code: {resp['Status']}")
stations = pd.DataFrame(resp["Data"])
stations.head()
stations_copy = stations.copy(deep=True)

In [4]:
#loop through all stations
size = 3000 #number of crashes to get
radius = 10 #radius in kms
crashes = {}
empty_stations = []
error_stations = []

for index, station in stations_copy.iterrows():
    # station_id = station['Station ID']
    station_id = "91306"
    crashes[station_id] = crashes.get(station_id, [])
    params = f"crashes/{station_id}/{size}/{radius}"
    resp = get_full_data(params)
    if resp is None or "Data" not in resp or "Token" not in resp or "Status" not in resp:
        error_stations.append(station_id)
        continue
    elif resp["Data"] == [] or resp["Token"] == "END" or resp["Status"] != 200:
        empty_stations.append(station_id)
        continue
    else:
        crashes[station_id].extend(resp["Data"])
        while resp and "Token" in resp and resp["Token"] != "END":
            params = f"stream/{resp['Token']}"
            resp = get_full_data(params)
            if resp and resp["Data"]:
                crashes[station_id].extend(resp["Data"])



Connection error: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Retrying in 5 seconds...
Connection error: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Retrying in 5 seconds...


KeyboardInterrupt: 

In [9]:
#loop through all stations
size = 3000 #number of crashes to get
radius = 10 #radius in kms
crashes = {}
empty_stations = []
error_stations = []

# for index, station in stations_copy.iterrows():
    # station_id = station['Station ID']
station_id = "91306"
crashes[station_id] = crashes.get(station_id, [])
params = f"crashes/{station_id}/{size}/{radius}"
resp = get_full_data(params)
if resp is None or "Data" not in resp or "Token" not in resp or "Status" not in resp:
    error_stations.append(station_id)
    pass
elif resp["Data"] == [] or resp["Token"] == "END" or resp["Status"] != 200:
    empty_stations.append(station_id)
    pass

else:
    crashes[station_id].extend(resp["Data"])
    while resp and "Token" in resp and resp["Token"] != "END":
        params = f"stream/{resp['Token']}"
        resp = get_full_data(params)
        if resp and resp["Data"]:
            crashes[station_id].extend(resp["Data"])



In [10]:
crashes_copy = copy.deepcopy(crashes)

In [11]:
#remove empty rows from crashes_copy
crashes_copy = {k: v for k, v in crashes_copy.items() if v}

#convert Dictionary to DataFrame
dfs = []
for key, value in crashes_copy.items():
  # Create a DataFrame from the list of dictionaries
    df = pd.json_normalize(value)
    #drop location and light_condition colunn
    df = df[['_source.crash_date', '_source.severity']]
    # Rename the columns
    df = df.rename(columns={'_source.crash_date': 'crash_date', '_source.severity': 'severity'})

    #format date to DD/MM/YYYY
    df['crash_date'] = pd.to_datetime(df['crash_date']).dt.strftime('%d/%m/%Y')
    
    # Add a new column 'key' with the key value
    df['Station ID'] = key
    # Add the DataFrame to the list
    dfs.append(df)

new_df = pd.concat(dfs, ignore_index=True)



In [13]:
new_df


,crash_date,severity,Station ID
0,27/09/2015,3,91306
1,06/03/2016,0,91306
2,18/05/2016,3,91306
3,24/01/2016,0,91306
4,04/06/2017,0,91306
...,...,...,...
94,08/03/2020,0,91306
95,19/12/2019,1,91306
96,26/04/2020,0,91306
97,01/07/2020,1,91306


In [17]:
#find minimum date
min_date = new_df['crash_date'].min()
#get year of minimum date
min_date = min_date.split('/')[2]
#find maximum date
max_date = new_df['crash_date'].max()
#get year of maximum date
max_date = max_date.split('/')[2]
print(min_date)
print(max_date)

2010
2017


In [18]:
#getting weather data from API
weather_data = []
for station in new_df['Station ID'].unique():
    params = f"weather/{station}/{min_date}/{max_date}"
    resp = get_full_data(params)
    if resp is None or "Data" not in resp or "Status" not in resp:
        error_stations.append(station)
        continue
    elif resp["Data"] == [] or resp["Status"] != 200:
        empty_stations.append(station)
        continue
    else:
        weather_data.extend(resp["Data"])

weather_data_copy = copy.deepcopy(weather_data)

weather_data_copy

91306


[{'UV': '32.98',
  'Max Humid': '81',
  'created_at': '2024-05-14T02:25:12.562449764Z',
  'Min Temp': '18.7',
  'WindSpeed': '4.36',
  'Min Humid': '17',
  'source': 'cressy_research_station',
  'Station Name': 'CRESSY RESEARCH STATION',
  'Date': '01/01/2010',
  'Rain': '0.0',
  'Pan-Rain': '-1.0',
  'Max Temp': '32.7',
  'Evapo-Rain': '8.8'},
 {'UV': '30.45',
  'Max Humid': '94',
  'created_at': '2024-05-14T02:25:12.562456586Z',
  'Min Temp': '13.4',
  'WindSpeed': '5.97',
  'Min Humid': '26',
  'source': 'cressy_research_station',
  'Station Name': 'CRESSY RESEARCH STATION',
  'Date': '02/01/2010',
  'Rain': '0.8',
  'Pan-Rain': '-1.0',
  'Max Temp': '24.6',
  'Evapo-Rain': '6.8'},
 {'UV': '32.39',
  'Max Humid': '78',
  'created_at': '2024-05-14T02:25:12.562462918Z',
  'Min Temp': '8.5',
  'WindSpeed': '4.43',
  'Min Humid': '28',
  'source': 'cressy_research_station',
  'Station Name': 'CRESSY RESEARCH STATION',
  'Date': '03/01/2010',
  'Rain': '0.0',
  'Pan-Rain': '-1.0',
  'Max

In [21]:
#convert Dictionary to DataFrame
weather_dfs = pd.DataFrame(weather_data_copy)
weather_dfs

,UV,Max Humid,created_at,Min Temp,WindSpeed,Min Humid,source,Station Name,Date,Rain,Pan-Rain,Max Temp,Evapo-Rain
0,32.98,81,2024-05-14T02:25:12.562449764Z,18.7,4.36,17,cressy_research_station,CRESSY RESEARCH STATION,01/01/2010,0.0,-1.0,32.7,8.8
1,30.45,94,2024-05-14T02:25:12.562456586Z,13.4,5.97,26,cressy_research_station,CRESSY RESEARCH STATION,02/01/2010,0.8,-1.0,24.6,6.8
2,32.39,78,2024-05-14T02:25:12.562462918Z,8.5,4.43,28,cressy_research_station,CRESSY RESEARCH STATION,03/01/2010,0.0,-1.0,23.5,6.4
3,34.36,93,2024-05-14T02:25:12.562469200Z,4.1,5.21,36,cressy_research_station,CRESSY RESEARCH STATION,04/01/2010,0.0,-1.0,24.8,6.5
4,32.08,92,2024-05-14T02:25:12.562475441Z,11.1,6.81,25,cressy_research_station,CRESSY RESEARCH STATION,05/01/2010,0.0,-1.0,25.7,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,8.22,96,2024-05-14T02:25:23.993933981Z,14.2,1.82,74,cressy_research_station,CRESSY RESEARCH STATION,28/12/2016,7.2,-1.0,24.5,2.1
2554,13.74,97,2024-05-14T02:25:23.993940163Z,16.7,3.52,63,cressy_research_station,CRESSY RESEARCH STATION,29/12/2016,8.8,-1.0,27.0,3.5
2555,17.71,97,2024-05-14T02:25:23.993946375Z,18.6,5.83,46,cressy_research_station,CRESSY RESEARCH STATION,30/12/2016,17.4,-1.0,24.6,4.7
2556,28.78,92,2024-05-14T02:25:23.993952546Z,12.1,5.33,48,cressy_research_station,CRESSY RESEARCH STATION,31/12/2016,0.0,-1.0,25.3,5.8


In [22]:
#drop created_at, source, Station Name columns
weather_dfs = weather_dfs.drop(['created_at', 'source', 'Station Name', 'Pan-Rain'], axis=1)
#drop rows with missing/magic values
#drop Min Temp, Max Temp	= -999
weather_dfs = weather_dfs[weather_dfs['Min Temp'] != -999.0]
#drop Rain, Evapo-Rain, Max Humid, Min Humid
weather_dfs['Station ID'] = station_id
weather_dfs

,UV,Max Humid,Min Temp,WindSpeed,Min Humid,Date,Rain,Pan-Rain,Max Temp,Evapo-Rain
0,32.98,81,18.7,4.36,17,01/01/2010,0.0,-1.0,32.7,8.8
1,30.45,94,13.4,5.97,26,02/01/2010,0.8,-1.0,24.6,6.8
2,32.39,78,8.5,4.43,28,03/01/2010,0.0,-1.0,23.5,6.4
3,34.36,93,4.1,5.21,36,04/01/2010,0.0,-1.0,24.8,6.5
4,32.08,92,11.1,6.81,25,05/01/2010,0.0,-1.0,25.7,7.4
...,...,...,...,...,...,...,...,...,...,...
2553,8.22,96,14.2,1.82,74,28/12/2016,7.2,-1.0,24.5,2.1
2554,13.74,97,16.7,3.52,63,29/12/2016,8.8,-1.0,27.0,3.5
2555,17.71,97,18.6,5.83,46,30/12/2016,17.4,-1.0,24.6,4.7
2556,28.78,92,12.1,5.33,48,31/12/2016,0.0,-1.0,25.3,5.8


In [26]:
merged_df = new_df.merge(weather_dfs, left_on=['Station ID', 'crash_date'], right_on=['Station ID', 'Date'], how='inner')
merged_df

,crash_date,severity,Station ID,UV,Max Humid,Min Temp,WindSpeed,Min Humid,Date,Rain,Pan-Rain,Max Temp,Evapo-Rain
0,27/09/2015,3,91306,15.08,100,-0.6,3.51,56,27/09/2015,0.0,-1.0,16.0,2.3
1,06/03/2016,0,91306,18.24,97,11.3,2.17,47,06/03/2016,0.0,-1.0,26.0,3.6
2,18/05/2016,3,91306,5.93,98,-999.0,4.26,71,18/05/2016,-1.0,-1.0,14.9,-1.0
3,24/01/2016,0,91306,23.35,86,-999.0,3.73,29,24/01/2016,-1.0,-1.0,28.0,-1.0
4,15/11/2013,3,91306,27.71,94,6.7,2.30,48,15/11/2013,0.6,-1.0,19.0,4.2
5,15/01/2015,3,91306,32.44,87,13.8,4.33,31,15/01/2015,0.0,-1.0,24.6,6.5
6,14/08/2016,1,91306,11.27,88,4.3,8.36,47,14/08/2016,0.2,-1.0,13.1,2.5
7,19/02/2010,0,91306,26.20,94,15.0,4.97,44,19/02/2010,0.0,-1.0,26.8,5.6
8,19/02/2010,0,91306,26.20,94,15.0,4.97,44,19/02/2010,0.0,-1.0,26.8,5.6
9,17/12/2010,0,91306,18.70,95,6.8,4.18,76,17/12/2010,0.0,-1.0,15.9,2.6
